In [ ]:
import pymysql.cursors
from collections import namedtuple

class ListCursor(pymysql.cursors.Cursor):
        def fetchall(self):
            ''' Fetch all the rows '''
            self._check_executed()
            if self._rows is None:
                return []
            if self.rownumber:
                result = [list(row) for row in self._rows[self.rownumber:]]
            else:
                result = [list(row) for row in self._rows]
            self.rownumber = len(self._rows)
            return result

class NamedTupleCursor(pymysql.cursors.Cursor):
        def fetchall(self):
            ''' Fetch all the rows '''
            self._check_executed()
            
            fields = ','.join([field.name for field in self._result.fields])
            Row = namedtuple('row', fields)
            
            if self._rows is None:
                return []
            if self.rownumber:
                result = [Row(*row) for row in self._rows[self.rownumber:]]
            else:
                result = [Row(*row) for row in self._rows]
            self.rownumber = len(self._rows)
            return result

connection1 = pymysql.connect(host='localhost',
                         user='webucator',
                         passwd='webuc8',
                         db='baseball',
                         charset='utf8mb4',
                         cursorclass=ListCursor)

connection2 = pymysql.connect(host='localhost',
                         user='webucator',
                         passwd='webuc8',
                         db='baseball',
                         charset='utf8mb4',
                         cursorclass=NamedTupleCursor)

query = '''SELECT year(debut) year, avg(height) height, avg(weight) weight
FROM master
WHERE debut is NOT NULL
GROUP BY year(debut)
ORDER BY year(debut)
LIMIT 5'''
    
with connection1.cursor() as cursor:
    cursor.execute(query)
    list_results = cursor.fetchall()
connection1.close()
    
with connection2.cursor() as cursor:
    cursor.execute(query)
    nt_results = cursor.fetchall()
connection2.close()

list_results, ' ', nt_results